In [ ]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [ ]:
#|export
# Python native modules
import os
import io
from typing import Tuple,Any,Optional,NamedTuple,Iterable
# Third party libs
import imageio
from fastcore.all import add_docs,ifnone
import matplotlib.pyplot as plt
import torchdata.datapipes as dp
from torchdata.datapipes import functional_datapipe
from IPython.core.display import Video,Image
from torchdata.dataloader2 import DataLoader2,MultiProcessingReadingService
# Local modules
from fastrl.loggers.core import LoggerBase,is_record
from fastrl.pipes.core import DataPipeAugmentationFn,apply_dp_augmentation_fns
from fastrl.loggers.jupyter_visualizers import ImageCollector

In [ ]:
#|default_exp loggers.vscode_visualizers

# Visualizers - VS-Code
> Iterable pipes for displaying environments as they run using `typing.NamedTuples` with `image` fields for VS-Code

`fastrl.jupyter_visualizers` can be used in vscode, however you likely will notice flickering for video
based outputs. For vscode, we can generate a gif instead.

In [ ]:
dp.iter.Repeater

In [ ]:
#|export
class SimpleVSCodeVideoPlayer(dp.iter.IterDataPipe):
    def __init__(self, 
                 source_datapipe=None, 
                 skip_frames:int=1,
                 fps:int=30,
                 downsize_res=(2,2)
        ):
        self.source_datapipe = source_datapipe
        self.fps = fps
        self.skip_frames = skip_frames
        self.downsize_res = downsize_res
        self._bytes_object = None
        self.frames = [] 

    def reset(self):
        super().reset()
        self._bytes_object = io.BytesIO()

    def show(self,start:int=0,end:Optional[int]=None,step:int=1):
        print(f'Creating gif from {len(self.frames)} frames')
        imageio.mimwrite(
            self._bytes_object,
            self.frames[start:end:step],
            format='GIF',
            duration=self.fps
        )
        return Image(self._bytes_object.getvalue())
        
    def __iter__(self) -> Tuple[NamedTuple]:
        n_frame = 0
        for record in self.source_datapipe:
            # for o in self.dequeue():
            if is_record(record) and record.name=='image':
                if record.value is None: continue
                n_frame += 1
                if n_frame%self.skip_frames!=0: continue
                self.frames.append(
                    record.value[::self.downsize_res[0],::self.downsize_res[1]]
                )
            yield record
add_docs(
SimpleVSCodeVideoPlayer,
"""Displays video from a `source_datapipe` that produces `typing.NamedTuples` that contain an `image` field.
This only can handle 1 env input.""",
show="In order to show the video, this must be called in a notebook cell.",
reset="Will reset the bytes object that is used to store file data."
)

In [ ]:
#|export
@functional_datapipe('visualize_vscode')
class VSCodeDataPipe(dp.iter.IterDataPipe):
    def __new__(self,source:Iterable):
        "This is the function that is actually run by `DataBlock`"
        pipe = ImageCollector(source).dump_records()
        pipe = SimpleVSCodeVideoPlayer(pipe)
        return pipe 
        

In [ ]:
from fastrl.envs.gym import GymDataPipe

In [ ]:
#|hide
pipe = GymDataPipe(['CartPole-v1'],None,n=100,seed=0,include_images=True).visualize_vscode()

list(pipe);
pipe.show()

In [ ]:
#|hide
pipe = GymDataPipe(['CartPole-v1'],None,n=100,seed=0,include_images=True)
pipe = VSCodeDataPipe(pipe)

list(pipe);
pipe.show()

In [ ]:
#|hide
#|eval: false
!nbdev_export